In [24]:
import xml.etree.ElementTree as ET
import pandas as pd


#Extract
tree = ET.parse("export.xml")
root = tree.getroot()


workout_data = []

#transform
for workout in root.findall("Workout"):
    workout_dict = {
        "activity_type": workout.get("workoutActivityType"),
        "duration_min": float(workout.get("duration", 0)),
        "source": workout.get("sourceName"),
        "start_date": workout.get("startDate"),
        "end_date": workout.get("endDate")
    }
    
    for stat in workout.findall("WorkoutStatistics"):
        stat_type = stat.get("type")  
        stat_sum = stat.get("sum")    
        stat_unit = stat.get("unit")  
        stat_average = stat.get("average")
    
        
        workout_dict[f"{stat_type}_sum"] = float(stat_sum) if stat_sum else 0
        workout_dict[f"{stat_type}_unit"] = stat_unit
        workout_dict[f"{stat_type}_average"] = float(stat_average) if stat_average else 0
       

    for metadata in workout.findall('MetadataEntry'): 
        metadata_key = metadata.get('key')
        metadata_value = metadata.get('value')

        workout_dict[f"{metadata_key}"] = metadata_value
        
    for event in workout.findall('WorkoutEvent'): 
        event_type = event.get('type')
        event_duration = event.get('duration')

        workout_dict[f"{event_type}_min"] = float(event_duration) if event_duration else 0
    
        for event_metadata in event.findall('MetadataEntry'): 
            event_metadata_key = metadata.get('key')
            event_metadata_value = metadata.get('value')

            workout_dict[f"{event_metadata_key}"] = event_metadata_value
 

    workout_data.append(workout_dict)


df = pd.DataFrame(workout_data)




In [25]:
df.head()

# # Save to CSV
df.to_csv("apple_workouts.csv", index=False)
print("Workout data extracted successfully!")

Workout data extracted successfully!


In [28]:
import google.oauth2 
import pandas as pd

# Google Cloud settings
PROJECT_ID = "[PROJECT_ID]"
DATASET_ID = "apple_workouts"
TABLE_ID = "workouts"

# Full table path
table_full_id = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# Upload DataFrame to BigQuery
df.to_gbq(table_full_id, project_id=PROJECT_ID, if_exists="replace")

print("Data uploaded successfully!")


/var/folders/w_/_0xjx5cs4pld6fr_8r2vk9wc0000gn/T/ipykernel_73327/4105096115.py:13: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(table_full_id, project_id=PROJECT_ID, if_exists="replace")


Data uploaded successfully!
